In [1]:
import tensorflow as tf
from tensorflow.python.platform import gfile
GRAPH_PB_PATH = './qw_w_scratch_1-0.bytes'
with tf.Session() as sess:
    print("load graph")
    with gfile.FastGFile(GRAPH_PB_PATH,'rb') as f:
        graph_def = tf.GraphDef()
    graph_def.ParseFromString(f.read())
    sess.graph.as_default()
    tf.import_graph_def(graph_def, name='')
    graph_nodes=[n for n in graph_def.node]
    names = []
    ops = []
    inputs = []
    for t in graph_nodes:
        names.append(t.name)
        ops.append(t.op)
        inputs.append(t.input)
    print(names)
    print(ops)
    #print(inputs)

/home/pi/berryconda3/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.4 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/home/pi/berryconda3/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: builtins.type size changed, may indicate binary incompatibility. Expected 432, got 412
  return f(*args, **kwds)


load graph
Instructions for updating:
Use tf.gfile.GFile.
[]
['Const', 'Identity', 'Placeholder', 'Const', 'Identity', 'Const', 'Identity', 'Sub', 'Cast', 'Const', 'Add', 'RealDiv', 'Sqrt', 'RealDiv', 'Const', 'Minimum', 'Const', 'Maximum', 'Const', 'Identity', 'Const', 'Identity', 'MatMul', 'BiasAdd', 'Sigmoid', 'Mul', 'Const', 'Identity', 'Const', 'Identity', 'MatMul', 'BiasAdd', 'Sigmoid', 'Mul', 'Const', 'Identity', 'Const', 'Identity', 'MatMul', 'BiasAdd', 'Sigmoid', 'Mul', 'Const', 'Identity', 'Const', 'Identity', 'MatMul', 'BiasAdd', 'Sigmoid', 'Mul', 'Const', 'Identity', 'Const', 'Identity', 'MatMul', 'BiasAdd', 'Const', 'Identity', 'Exp', 'Shape', 'Const', 'Const', 'RandomStandardNormal', 'Mul', 'Add', 'Sqrt', 'Mul', 'Add', 'Const', 'Minimum', 'Const', 'Maximum', 'Const', 'RealDiv', 'Identity', 'StopGradient', 'Sub', 'Square', 'Const', 'Mul', 'RealDiv', 'Const', 'Log', 'Const', 'Mul', 'Sub', 'Const', 'Mul', 'Sub', 'Identity', 'Const', 'Identity', 'Const', 'Identity', 'MatMul',

In [2]:
def load_graph(frozen_graph_filename):
    # We load the protobuf file from the disk and parse it to retrieve the 
    # unserialized graph_def
    with tf.gfile.GFile(frozen_graph_filename, "rb") as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())

    # Then, we import the graph_def into a new Graph and returns it 
    with tf.Graph().as_default() as graph:
        # The name var will prefix every op/nodes in your graph
        # Since we load everything in a new graph, this is not needed
        tf.import_graph_def(graph_def, name="prefix")
    return graph

In [3]:
# We use our "load_graph" function
graph = load_graph(GRAPH_PB_PATH)

# We can verify that we can access the list of operations in the graph
#for op in graph.get_operations():
    #print([op.name,op.values()])
    # prefix/Placeholder/inputs_placeholder
    # ...
    # prefix/Accuracy/predictions
    
# We access the input and output nodes 
x = graph.get_tensor_by_name('prefix/vector_observation:0')
y = graph.get_tensor_by_name('prefix/action:0')

# We launch a Session
# with tf.Session(graph=graph) as sess:
#     # Note: we don't nee to initialize/restore anything
#     # There is no Variables in this graph, only hardcoded constants 
#     y_out = sess.run(y, feed_dict={
#         x: [[3, 5, 7, 4, 5, 1, 1, 1, 1, 1,3, 5, 7, 4, 5, 1, 1, 1, 1, 1,3, 5, 7, 4, 5, 1]] # < 45
#     })
#     # I taught a neural net to recognise when a sum of numbers is bigger than 45
#     # it should return False in this case
#     #print(y_out) # [[ False ]] Yay, it works!

In [4]:
resultsfile = "state_history.txt"
thrusts = []
states = []
with open(resultsfile) as f:
        for line in f:
            res = eval(line)
            thrusts.append(res["thrusts"])
            states.append(res["states"])

In [18]:
import numpy as np
import time
pred_thrusts = []
with tf.Session(graph=graph) as sess:
    for state in states:
        #x_feed = np.random.rand(26)
        #x_feed = [0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,-1,0,1,0,-1]
        y_out = sess.run(y, feed_dict={
            x: [state]
        })
        pred_thrusts.append(y_out)
        #print(y_out[0][0:4])

In [22]:
thrusts = np.asarray(thrusts)
pred_thrusts = np.asarray(pred_thrusts)
errs = []
for i in range(len(pred_thrusts)):
    errs.append(np.linalg.norm(thrusts[i]-pred_thrusts[i])/4)
